In [1]:
!pip3 install tensorflow-gpu

     |████████████████████████████████| 320.4MB 53kB/s 


In [2]:
import tensorflow as tf
tf.__version__

'2.3.1'

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from numpy import argmax,array
from tensorflow.keras.utils import plot_model,to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input,Dense,LSTM,Embedding,Dropout
from tensorflow.keras.layers import Add
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
from tensorflow.keras.preprocessing.image import load_img,img_to_array
import pickle
from pickle import load
from pickle import dump
from PIL import Image

from keras.preprocessing.text import Tokenizer
from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu

from keras.layers.merge import add

In [5]:
def load_doc(filename):
    file = open(filename,'r')
    text = file.read()
    file.close()    
    return text

def load_set(filename):
    doc = load_doc(filename)
    dataset = list()
    
    for line in doc.split('\n'):
        if len(line)<1:
            continue
        identifier = line.split('.')[0]
        dataset.append(identifier)
    return set(dataset)

def load_clean_descriptions(filename, dataset):
    doc = load_doc(filename)
    descriptions = dict()
    
    for line in doc.split('\n'):
        tokens = line.split()
        image_id, image_desc = tokens[0],tokens[1:]
        #if image_id not in dataset ignore
        if image_id in dataset:
            if image_id not in descriptions:
                descriptions[image_id] = list()
            desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
            descriptions[image_id].append(desc)
    return descriptions

def load_photo_features(filename, dataset):
	all_features = pickle.load(open(filename, 'rb'))
	features = {k: all_features[k] for k in dataset}
	return features

In [6]:
def word_for_id(integer,tokenizer):
  for word,index in tokenizer.word_index.items():
    if index==integer:
      return word
  return None

def data_generator(descriptions, photos, tokenizer, max_length, vocab_size):
	while 1:
		for key, desc_list in descriptions.items():
			photo = photos[key][0]
			in_img, in_seq, out_word = create_sequences(tokenizer, max_length, desc_list, photo, vocab_size)
			yield [in_img, in_seq], out_word


def generate_desc(model, tokenizer, photo, max_length):
    in_text = 'startseq'
    for i in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence],maxlen = max_length)
        yhat = model.predict([photo,sequence],verbose=0)
        yhat = argmax(yhat)
        word = word_for_id(yhat,tokenizer)
        if word is None:
            break

        in_text += ' '+word
        if word == 'endseq':
            break	
    return in_text

In [7]:
def create_sequences(tokenizer, max_length, desc_list, photo, vocab_size):
	X1, X2, y = list(), list(), list()
	for desc in desc_list:
		seq = tokenizer.texts_to_sequences([desc])
		for i in range(1, len(seq[0])):
			in_seq, out_seq = seq[0][:i], seq[0][i]
			in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
			out_seq = to_categorical([out_seq], num_classes=vocab_size)
			X1.append(photo)
			X2.append(in_seq)
			y.append(out_seq.flatten())
	return array(X1), array(X2), array(y)

In [8]:
def to_lines(descriptions):
    all_desc = list()
    for key in descriptions.keys():
        [all_desc.append(des) for des in descriptions[key]]
    return all_desc

def create_tokenizer(descriptions):
    lines = to_lines(descriptions)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer    

def find_max_length(descriptions):
	lines = to_lines(descriptions)
	return max(len(d.split()) for d in lines)

In [9]:
def evaluate_model(model, descriptions, photos, tokenizer, max_length):
    actual, predicted = list(), list()
    
    for key, desc_list in descriptions.items():
            yhat = generate_desc(model, tokenizer, photos[key], max_length)
            references = [d.split() for d in desc_list]
            actual.append(references)
            predicted.append(yhat.split())
    print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
    print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
    print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
    print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

In [11]:
filename = '/content/drive/My Drive/College/IRS Innovative/flicker8k-dataset/Flickr8k_text/Flickr_8k.trainImages.txt'
train = load_set(filename)
filename = '/content/drive/My Drive/College/IRS Innovative/flicker8k-dataset/Flickr8k_text/Flickr_8k.devImages.txt'
test = load_set(filename)

In [12]:
all = train.union(test)
descriptions = load_clean_descriptions('/content/drive/My Drive/College/IRS Innovative/flicker8k-dataset/Flickr8k_text/descriptions.txt', all)
tokenizer = create_tokenizer(descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)

# determine the maximum sequence length
max_length = find_max_length(descriptions)
print('Description Length: %d' % max_length)

Vocabulary Size: 8170
Description Length: 34


In [14]:
testImgList ='/content/drive/My Drive/College/IRS Innovative/flicker8k-dataset/Flickr8k_text/Flickr_8k.testImages.txt'
testImg = load_set(testImgList)
print('Dataset: %d' % len(testImg))
# descriptions
test_descriptions = load_clean_descriptions('/content/drive/My Drive/College/IRS Innovative/flicker8k-dataset/Flickr8k_text/descriptions.txt', testImg)
print('Descriptions: test=%d' % len(test_descriptions))
# photo features
test_features = load_photo_features('/content/drive/My Drive/College/IRS Innovative/flicker8k-dataset/Flickr8k_Dataset/features_resnet.pkl', testImg)
print('Photos: test=%d' % len(test_features))

weights = '/content/drive/My Drive/College/IRS Innovative/ResNet50/model_19.h5'
model = load_model(weights)
evaluate_model(model, test_descriptions, test_features, tokenizer, max_length)

# prepare tokenizer
# tokenizer = create_tokenizer(train_descriptions)
# save the tokenizer
# dump(tokenizer, open(code_root_dir + 'tokenizer_resnet50.pkl', 'wb'))


Dataset: 1000
Descriptions: test=1000
Photos: test=1000
BLEU-1: 0.502172
BLEU-2: 0.256557
BLEU-3: 0.175373
BLEU-4: 0.081145
